# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments          Customer  \
0  Dec 22 2022 11:08AM  253672        10  PRF-45318           Bio-PRF   
1  Dec 22 2022 11:08AM  253672        10  PRF-45325           Bio-PRF   
2  Dec 22 2022 11:08AM  253672        10  PRF-45334           Bio-PRF   
3  Dec 22 2022 11:08AM  253672        10  PRF-45337           Bio-PRF   
4  Dec 22 2022 11:08AM  253672        10  PRF-45344           Bio-PRF   
5  Dec 22 2022 11:08AM  253672        10  PRF-45377           Bio-PRF   
6  Dec 22 2022 11:08AM  253672        10  PRF-45380           Bio-PRF   
7  Dec 22 2022 11:07AM  253671        10    8931051  Eywa Pharma Inc.   
8  Dec 22 2022 11:07AM  253671        10    8931052  Eywa Pharma Inc.   
9  Dec 22 2022 11:07AM  253671        10    8931053  Eywa Pharma Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  253667       Released          4
23  253668       Released          1
24  253669       Released          3
25  253671       Released         14
26  253672       Released          7

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
253667                NaN       4.0
253668                NaN       1.0
253669                NaN       3.0
253671                NaN      14.0
253672                NaN       7.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
253590                0.0       1.0
253603                1.0       0.0
253622                0.0       1.0
253624                0.0       1.0
253625                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
253590                  0         1
253603                  1         0
253622                  0         1
253624                  0         1
253625                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               253590          0         1
1               253603          1         0
2               253622          0         1
3               253624          0         1
4               253625          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               253590                  1
1               253603         1         
2               253622                  1
3               253624                  1
4               253625                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                Customer
0   Dec 22 2022 11:08AM  253672        10                 Bio-PRF
7   Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.
21  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.
24  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.
25  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC
29  Dec 22 2022 10:24AM  253666        15   Alliance Pharma, Inc.
41  Dec 22 2022  9:22AM  253664        10       ISDIN Corporation
44  Dec 22 2022  9:22AM  253665        10       ISDIN Corporation
81  Dec 22 2022  9:05AM  253661        10       ISDIN Corporation
94  Dec 22 2022  8:53AM  253660        10        Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                Customer Executing  \
0  Dec 22 2022 11:08AM  253672        10                 Bio-PRF             
1  Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.             
2  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.             
3  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.             
4  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC             
5  Dec 22 2022 10:24AM  253666        15   Alliance Pharma, Inc.         2   
6  Dec 22 2022  9:22AM  253664        10       ISDIN Corporation             
7  Dec 22 2022  9:22AM  253665        10       ISDIN Corporation             
8  Dec 22 2022  9:05AM  253661        10       ISDIN Corporation             
9  Dec 22 2022  8:53AM  253660        10        Methapharm, Inc.             

  Released  
0        7  
1       14  
2        3  
3        1  
4        4  
5       10  
6        3  
7       37  
8       13  
9        9

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                Customer Released  \
0  Dec 22 2022 11:08AM  253672        10                 Bio-PRF        7   
1  Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.       14   
2  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.        3   
3  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.        1   
4  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC        4   
5  Dec 22 2022 10:24AM  253666        15   Alliance Pharma, Inc.       10   
6  Dec 22 2022  9:22AM  253664        10       ISDIN Corporation        3   
7  Dec 22 2022  9:22AM  253665        10       ISDIN Corporation       37   
8  Dec 22 2022  9:05AM  253661        10       ISDIN Corporation       13   
9  Dec 22 2022  8:53AM  253660        10        Methapharm, Inc.        9   

  Executing  
0            
1            
2            
3            
4            
5         2  
6            
7            
8            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                Customer Released  \
0  Dec 22 2022 11:08AM  253672        10                 Bio-PRF        7   
1  Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.       14   
2  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.        3   
3  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.        1   
4  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC        4   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                Customer  Released  \
0  Dec 22 2022 11:08AM  253672        10                 Bio-PRF       7.0   
1  Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.      14.0   
2  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.       3.0   
3  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.       1.0   
4  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC       4.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                Customer  Released  \
0  Dec 22 2022 11:08AM  253672        10                 Bio-PRF       7.0   
1  Dec 22 2022 11:07AM  253671        10        Eywa Pharma Inc.      14.0   
2  Dec 22 2022 10:53AM  253669        10          Citieffe, Inc.       3.0   
3  Dec 22 2022 10:47AM  253668        21       NBTY Global, Inc.       1.0   
4  Dec 22 2022 10:33AM  253667        15  Mizner Bioscience, LLC       4.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3804770     104.0        1.0
12          253654      16.0        0.0
15          507333      14.0        2.0
17          760877       3.0        0.0
19          507284      36.0        0.0
20          253590       1.0        0.0
21          507271       1.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3804770     104.0        1.0
1        12   253654      16.0        0.0
2        15   507333      14.0        2.0
3        17   760877       3.0        0.0
4        19   507284      36.0        0.0
5        20   253590       1.0        0.0
6        21   507271       1.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     104.0        1.0
1        12      16.0        0.0
2        15      14.0        2.0
3        17       3.0        0.0
4        19      36.0        0.0
5        20       1.0        0.0
6        21       1.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  104.0
1        12  Released   16.0
2        15  Released   14.0
3        17  Released    3.0
4        19  Released   36.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   17    19   20   21
Status                                           
Executing    1.0   0.0   2.0  0.0   0.0  0.0  1.0
Released   104.0  16.0  14.0  3.0  36.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   17    19   20   21
0          Executing    1.0   0.0   2.0  0.0   0.0  0.0  1.0
1           Released  104.0  16.0  14.0  3.0  36.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   17    19   20   21
0  Executing    1.0   0.0   2.0  0.0   0.0  0.0  1.0
1   Released  104.0  16.0  14.0  3.0  36.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()